In [1]:
import polars as pl

In [37]:
facility_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_data.parquet')
ballot_measures_df = pl.read_csv('../../003_data/001_raw-data/Ballot Measures Data Clean.csv')



In [38]:
ballot_measures_df

,county name,yes count 2020,no count 2020,total count 2020,yes perc 2020,no perc 2020,yes count 2018,no count 2018,total count 2018,yes perc 2018,no perc 2018,yes count 2022,no count 2022,total count 2022,yes perc 2022,no perc 2022
i64,str,f64,f64,f64,f64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64
0,"""Alameda""",329873.0,413277.0,743150.0,0.443885,0.556115,275550,280735,556285,0.49534,0.50466,182697.0,290746.0,473443.0,0.38589,0.61411
1,"""Alpine""",344.0,369.0,713.0,0.482468,0.517532,298,281,579,0.51468,0.48532,272.0,317.0,589.0,0.4618,0.5382
2,"""Amador""",4615.0,17149.0,21764.0,0.212047,0.787953,5256,11775,17031,0.308614,0.691386,3590.0,14569.0,18159.0,0.197698,0.802302
3,"""Butte""",29338.0,70174.0,99512.0,0.294819,0.705181,30908,55394,86302,0.358138,0.641862,16478.0,54015.0,70493.0,0.233754,0.766246
4,"""Calaveras""",5538.0,20896.0,26434.0,0.209503,0.790497,6688,14224,20912,0.319816,0.680184,4280.0,16440.0,20720.0,0.206564,0.793436
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
53,"""Tulare""",44506.0,99331.0,143837.0,0.30942,0.69058,28109,69542,97651,0.287852,0.712148,22954.0,66604.0,89558.0,0.256303,0.743697
54,"""Tuolumne""",6541.0,23103.0,29644.0,0.220652,0.779348,7554,15546,23100,0.327013,0.672987,4746.0,17874.0,22620.0,0.209814,0.790186
55,"""Ventura""",140977.0,270965.0,411942.0,0.342225,0.657775,114221,187357,301578,0.378744,0.621256,85257.0,187577.0,272834.0,0.312487,0.687513


In [39]:
ballot_measures_df = ballot_measures_df.rename({"county name": "county"})

ballot_measures_df = (
    ballot_measures_df
    .unpivot(
        index = ['county'],
        on = ['yes count 2022', 'no count 2022' ,'yes count 2020', 'no count 2020', 'yes count 2018', 'no count 2018'],
        value_name = "vote_count"
    )
    .with_columns(
    pl.col("variable")
    .str.split_exact(" count ", 1)
    .struct.rename_fields(["vote_type", "year"])
    .alias("fields")
    ).unnest("fields")
    .drop("variable")
)



In [41]:
ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("year")
        .str.to_date("%Y")
    )
)

ballot_measures_df

county,vote_count,vote_type,year
str,f64,str,date
"""Alameda""",182697.0,"""yes""",2022-01-01
"""Alpine""",272.0,"""yes""",2022-01-01
"""Amador""",3590.0,"""yes""",2022-01-01
"""Butte""",16478.0,"""yes""",2022-01-01
"""Calaveras""",4280.0,"""yes""",2022-01-01
…,…,…,…
"""Tulare""",69542.0,"""no""",2018-01-01
"""Tuolumne""",15546.0,"""no""",2018-01-01
"""Ventura""",187357.0,"""no""",2018-01-01


In [42]:
facility_df.head()

provider_number,network,facility_name,five_star_date,five_star,five_star_data_availability_code,address_line_1,address_line_2,state,zip_code,profit_or_nonprofit,chain_owned,chain_organization,late_shift,_of_dialysis_stations,offers_incenter_hemodialysis,offers_peritoneal_dialysis,offers_home_hemodialysis_training,certification_date,claims_date,eqrs_date,smr_date,patient_survival_category_text,patient_survival_data_availability_code,number_of_patients_included_in_survival_summary,mortality_rate_facility,mortality_rate_upper_confidence_limit_975,mortality_rate_lower_confidence_limit_25,shr_date,patient_hospitalization_category_text,patient_hospitalization_data_availability_code,number_of_patients_included_in_hospitalization_summary,hospitalization_rate_upper_confidence_limit_975,hospitalization_rate_lower_confidence_limit_25,srr_date,patient_hospital_readmission_category,patient_hospital_readmission_data_availability_code,…,percentage_of_adult_patients_with_serum_phosphorus_between_5670_mgdl,percentage_of_adult_patients_with_serum_phosphorus_greater_than_70_mgdl,long_term_catheter_data_availability_code,number_of_patients_in_long_term_catheter_summary,number_of_patient_months_in_long_term_catheter_summary,percentage_of_adult_patients_with_long_term_catheter_in_use,npcr_data_availability_code,number_of_patients_in_npcr_summary,number_of_patientmonths_in_npcr_summary,percentage_of_pediatric_hd_patients_with_npcr,year,month,city,county,phone_number,certification_or_recertification_date,crownweb_date,serum_phosphorus_data_availability_code_,dateswr,offers_incenter_peritoneal_dialysis,rate_of_hospital_readmission_category_text,percentage_of_medicare_patients_with_hgb_10_gdl,number_of_patients_included_in_transfusion_summary,standard_infection_ratio_,percentage_of_adult_hd_patients_with_ktv_12,percentage_of_adult_pd_patients_with_ktv17,percentage_of_pediatric_hd_patents_with_ktv12,percentage_of_pediatric_pd_patents_with_ktv18,number_of_adult_patients_included_in_arterial_venous_fistula_and_catheter_summaries,number_of_adult_patientmonths_included_in_arterial_venous_fistula_and_catheter_summaries,arteriovenous_fistulae_in_use_data_availability_code,percentage_of_patients_with_arteriovenous_fistulae_in_use,vascular_catheter_data_availability_code,percentage_of_patients_with_vascular_catheter_in_use_for_90_days_or_longer,hospitalization_rate_facility_,patient_hospital_readmission_category_text,readmission_rate_facility_
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""52305""","""17""","""SANTA CLARA VALLEY RENAL CARE …","""01Jan2019-31Dec2022""","""3""","""1""","""2220 MOORPARK AVENUE""",null,"""CA""","""95128""","""non-profit""","""no""","""Independent""","""yes""","""25""","""yes""","""yes""","""no""","""22AUG1977""","""01OCT2022-30SEP2023""","""01OCT2022-30SEP2023""","""01Jan2019-31Dec2022""","""As Expected""","""1""","""529""","""19.8""","""30.5""","""13.3""","""01Jan2022-31Dec2022""","""As Expected""","""1""","""117""","""205.7""","""97.9""","""01Jan2022-31Dec2022""","""As Expected""","""1""",…,"""31""","""11""","""1""","""192""","""1771""","""22""","""259""","""0""",null,null,"""2024""","""7""","""SAN JOSE""","""Santa Clara""","""(408) 885-5730""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""52311""","""18""","""St. Joseph Hospital Renal Cent…","""01Jan2019-31Dec2022""","""3""","""1""","""Sr. Elizabeth Bldg. 1100 W. St…",null,"""CA""","""92868""","""non-profit""","""no""","""Independent""","""yes""","""39""","""yes""","""yes""","""yes""","""15AUG1977""","""01OCT2022-30SEP2023""","""01OCT2022-30SEP2023""","""01Jan2019-31Dec2022""","""As Expected""","""1""","""533""","""22.9""","""33.7""","""16.1""","""01Jan2022-31Dec2

In [46]:

# It seems like dialysis facilites aren't located in more rural parts of california

facility_df = (
    facility_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county")
    )
)

# Get unique counties from both dataframes
ballot_counties = ballot_measures_df['county'].unique().sort()
facility_counties = facility_df['county'].unique().sort()

print("Unique counties in ballot_measures_df:")
print(ballot_counties)

print("\nUnique counties in facility_df:")
print(facility_counties)

# Find counties that are in one dataframe but not the other
counties_only_in_ballot = set(ballot_counties) - set(facility_counties)
counties_only_in_facility = set(facility_counties) - set(ballot_counties)

print("\nCounties only in ballot_measures_df:")
print(counties_only_in_ballot)

print("\nCounties only in facility_df:")
print(counties_only_in_facility)

# Check for case differences
case_differences = [c for c in ballot_counties if c not in facility_counties and c.lower() in [fc.lower() for fc in facility_counties]]

print("\nCounties with case differences:")
print(case_differences)

counties_only_in_ballot = set(ballot_counties) - set(facility_counties)

print("\nCounties only in ballot_measures_df:")
print(counties_only_in_ballot)



Unique counties in ballot_measures_df:
shape: (58,)
Series: 'county' [str]
[
	"alameda"
	"alpine"
	"amador"
	"butte"
	"calaveras"
	…
	"tulare"
	"tuolumne"
	"ventura"
	"yolo"
	"yuba"
]

Unique counties in facility_df:
shape: (48,)
Series: 'county' [str]
[
	""
	"alameda"
	"butte"
	"columbia"
	"colusa"
	…
	"tulare"
	"tuolumne"
	"ventura"
	"yolo"
	"yuba"
]

Counties only in ballot_measures_df:
{'siskiyou', 'amador', 'modoc', 'alpine', 'sierra', 'plumas', 'glenn', 'calaveras', 'trinity', 'mariposa', 'mono', 'lassen'}

Counties only in facility_df:
{'', 'columbia'}

Counties with case differences:
[]

Counties only in ballot_measures_df:
{'siskiyou', 'amador', 'modoc', 'alpine', 'sierra', 'plumas', 'glenn', 'calaveras', 'trinity', 'mariposa', 'mono', 'lassen'}
